In [1]:
import os 
os.chdir('../')

In [2]:
pwd

'/Users/hitam/Downloads/EndToEnd_MLProject_With_CiCd-Docker'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    train_data_path: Path
    test_data_path : Path
    transformer_name : Path
    model_name: str
    alpha: float
    target_column: str

In [4]:
from BostonHousePrediction.constants import *
from BostonHousePrediction.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RidgeRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            transformer_name = config.transformer_name,
            model_name = config.model_name,
            alpha = params.alpha,
            target_column = schema.name
            
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os
from BostonHousePrediction import logger
from sklearn.linear_model import Ridge
import joblib

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = Ridge(alpha=self.config.alpha,random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-21 20:13:47,038: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-21 20:13:47,070: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-21 20:13:47,086: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-21 20:13:47,089: INFO: common: created directory at: artifacts]
[2024-01-21 20:13:47,093: INFO: common: created directory at: artifacts/model_trainer]
